In [1]:
import pandas as pd
import numpy as np

import fsspec
import pandas as pd
import numpy as np
import datetime

import pandas as pd
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")

In [2]:
brand = 'LACOSTE'
country = 'UAE'
gcs_path = "gs://trd-sf-ntb"
target="units"
experiment_name = "model_pipeline"
cutoff_month="2025-07"
file_name = "store_features.parquet"

In [3]:
pd.read_pickle("active_stores.pkl")

0     52003
1     52009
2     52010
3     52012
4     52020
5     52043
6     52052
7     52071
8     52074
9     52082
10    52085
11    52086
dtype: object

In [4]:
# Define list of store IDs
lacoste_store_ids = pd.read_pickle("active_stores.pkl").to_list()

# Convert list to comma-separated string for SQL IN clause
faces_store_ids_str = ", ".join(str(sid) for sid in lacoste_store_ids)

In [5]:
from google.cloud import bigquery

# Initialize BigQuery client
client = bigquery.Client()

# Define the SQL query
query_stores = f"""
SELECT *
FROM `chb-svc-tredence-d001.shared_analytics_prod.dim_retail_loc`
WHERE store IN ({faces_store_ids_str}) 
"""

# Run query and convert to DataFrame
query_job_stores = client.query(query_stores)
stores_df = query_job_stores.to_dataframe()

In [6]:
stores_df.head(2)

,chain,chain_name,area,area_name,region,region_name,district,district_name,channel_id,channel_name,...,post,country_desc,description,is_ecom_fullfillment_loc,channel,developer,type_of_store,gla,latitude,longitude
0,2,Retail,2,UAE,2,ALLIED ENTERPRISES LLC,26,LACOSTE,3,Fashion,...,None,United Arab Emirates,Abu Dhabi,True,Retail,Aswaq Management and Services (AMS Real Estate...,Store in mall,202.99,24.4959241,54.3832259
1,2,Retail,2,UAE,2,ALLIED ENTERPRISES LLC,26,LACOSTE,3,Fashion,...,None,United Arab Emirates,Abu Dhabi,True,Retail,Aldar Properties,Store in mall,158,24.4888196,54.6086898


In [7]:
stores_df.columns

Index(['chain', 'chain_name', 'area', 'area_name', 'region', 'region_name',
       'district', 'district_name', 'channel_id', 'channel_name',
       'channel_type', 'store', 'store_name', 'store_name10', 'store_name3',
       'mall_name', 'store_open_date', 'store_close_date', 'total_square_ft',
       'selling_square_ft', 'vat_region', 'store_format', 'currency_code',
       'addr_key', 'module', 'key_value_1', 'key_value_2', 'addr_type',
       'primary_addr_ind', 'add_1', 'add_2', 'city', 'state', 'country_id',
       'post', 'country_desc', 'description', 'is_ecom_fullfillment_loc',
       'channel', 'developer', 'type_of_store', 'gla', 'latitude',
       'longitude'],
      dtype='object')

In [8]:
# selling_square_ft                  0
# store_format                       0
# type_of_store                      0
# is_ecom_fullfillment_loc           0
# state                              0
# channel                    

In [9]:
stores_df[["type_of_store","state","store"]]

,type_of_store,state,store
0,Store in mall,AZ,52010
1,Store in mall,AZ,52085
2,Store in mall,DU,52043
3,Store in mall,DU,52020
4,Store in mall,DU,52003
5,Store in mall,DU,52074
6,None,DU,52052
7,None,DU,52086
8,None,DU,52082
9,Store in mall,DU,52071


In [10]:
recommended_features = [ 'store', 'city','is_ecom_fullfillment_loc','channel',
                        'total_square_ft','selling_square_ft', 'store_format']

stores_df=stores_df[recommended_features].fillna(0)
print(f"selected features:\n{recommended_features}")
quant_features=["total_square_ft","selling_square_ft"]
qualit_features=recommended_features.copy()
[qualit_features.remove(i) for i in quant_features if i in qualit_features]
for col in quant_features:
    stores_df[col]=stores_df[col].astype(float)

print(f"quantitave features:\n{quant_features}")

print(f"qualitative features:\n{qualit_features}")


selected features:
['store', 'city', 'is_ecom_fullfillment_loc', 'channel', 'total_square_ft', 'selling_square_ft', 'store_format']
quantitave features:
['total_square_ft', 'selling_square_ft']
qualitative features:
['store', 'city', 'is_ecom_fullfillment_loc', 'channel', 'store_format']


In [11]:
stores_df[quant_features]

,total_square_ft,selling_square_ft
0,203.0,163.0
1,158.0,121.0
2,230.0,194.0
3,142.0,113.0
4,159.0,144.0
5,158.0,135.0
6,35.0,35.0
7,0.0,0.0
8,0.0,0.0
9,242.0,205.0


In [12]:
stores_df["total_square_ft_cat"]=pd.qcut(stores_df["total_square_ft"],q=4,labels = [1,2,3,4])
stores_df["selling_square_ft_cat"]=pd.qcut(stores_df["selling_square_ft"],q=4,labels = [1,2,3,4])

In [13]:
stores_df

,store,city,is_ecom_fullfillment_loc,channel,total_square_ft,selling_square_ft,store_format,total_square_ft_cat,selling_square_ft_cat
0,52010,ABU DHABI,True,Retail,203.0,163.0,1.000000000,3,3
1,52085,ABU DHABI,True,Retail,158.0,121.0,1.000000000,2,2
2,52043,DUBAI,True,Retail,230.0,194.0,2.000000000,4,4
3,52020,DUBAI,False,Retail,142.0,113.0,1.000000000,2,2
4,52003,DUBAI,True,Retail,159.0,144.0,1.000000000,3,3
5,52074,DUBAI,True,Retail,158.0,135.0,1.000000000,2,3
6,52052,DUBAI,False,Ecommerce,35.0,35.0,5.000000000,1,1
7,52086,DUBAI,False,Farfetch,0.0,0.0,5.000000000,1,1
8,52082,DUBAI,False,Farfetch,0.0,0.0,5.000000000,1,1
9,52071,DUBAI,True,Retail,242.0,205.0,1.000000000,4,4


In [14]:

# Construct full dynamic path

full_path = f"{gcs_path}/{target}/{brand}/{experiment_name}/{cutoff_month}/{file_name}"

In [15]:
full_path

'gs://trd-sf-ntb/units/LACOSTE/model_pipeline/2025-07/store_features.parquet'

In [16]:
stores_df.to_parquet(full_path,index=False)

In [17]:
hello

NameError: name 'hello' is not defined

In [ ]:
def eda_features(df_input: pd.DataFrame, feature_name: str):
    '''
    feature_name can only be following : temporal, promotion, marketing, store
    '''
    print(f"------EDA on {feature_name} features------------------")
    try:
        print(f"min & max dates {df_input.date.min().date()},{df_input.date.max().date()}",end="\n\n")
    except:
        pass
    print(f"shape of dataset : {df_input.shape}",end="\n\n")
    print(f"{feature_name} features: {df_input.columns.values}",end="\n\n")
    print(f"missing values :\n{df_input.isnull().sum()}")
    

In [ ]:
eda_features(stores_df,"stores")